In [ ]:
import torch
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# GPU 연산 정의 및 MNIST 데이터 셋 불러오기
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset = torchvision.datasets.MNIST("../data/", download=True, train=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(dataset, batch_size=50, shuffle=True)

In [ ]:
# 모델 구축
# 기본 오토인코더 모델은 층을 여러 개 쌓았다고 해서 스택 오토인코더라고도 불린다.
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 10),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(10, 32),
            nn.ReLU(),
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.Linear(128, 28*28),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [ ]:
# 모델, 손실 함수, 최적화 기법 정의하기
model = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# 학습하기
for epoch in range(51):
    running_loss = 0.0
    for data in trainloader:
        inputs = data[0].to(device)
        optimizer.zero_grad()
        outputs = model(inputs.view(-1, 28*28)) # 이미지를 일렬로 펴서 넣어주기 위해 inputs.view(-1, 28*28)을 입력값으로 사용한다.
        outputs = outputs.view(-1, 1, 28, 28) # 벡터 형태로 나온 출력값을 다시 정사각형 이미지로 변환하기 위해 outputs.view(-1, 1, 28, 28)을 작성한다.
        loss = criterion(inputs, outputs)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    cost = running_loss / len(trainloader)
    print("[%d] loss : %.3f" %(epoch+1, cost))

In [ ]:
# 디노이징 오토인코더
# 디노이징 오토인코더는 스택 오토인코더(오토인코더)와 동일하며, 입력 이미지에 가우시안 노이즈만 주입해주면 된다.
for epoch in range(101):
    running_loss = 0.0
    for data in trainloader:
        inputs = data[0].to(device)
        optimizer.zero_grad()
        dirty_inputs = inputs + torch.normal(0, 0.5, size=inputs.size()).to(device)
        outputs = model(dirty_inputs.view(-1, 28*28))
        outputs = outputs.view(-1, 1, 28, 28)
        loss = criterion(inputs, outputs)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    cost = running_loss / len(trainloader)
    print("[%d] loss:%.3f" %(epoch+1, cost))

In [ ]:
# 합성곱 오토인코더
# nn.Linaer() 대신 합성곱 층 nn.Conv2d를 사용하는 구조이다.
# 즉, 이미지 데이터가 일렬로 펴지지 않고 그대로 들어와 연산이 진행된다.

# 피처맵 벡터화
class Flatten(torch.nn.Module):
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)

In [ ]:
# 벡터를 사각형 피쳐맵으로 변환
class Deflatten(nn.Module):
    def __init__(self, k):
        super(Deflatten, self).__init__()
        self.k = k
    
    def forward(self, x):
        s = x.size()
        feature_size = int((s[1] // self.k)**.5)
        return x.view(s[0], self.k, feature_size, feature_size)

In [ ]:
# 모델 구축하기
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        k = 16
        self.encoder = nn.Sequential(
            nn.Conv2d(1, k, 3, stride=2), nn.ReLU(),
            nn.Conv2d(k, 2*k, 3, stride=2), nn.ReLU(),
            nn.Conv2d(2*k, 4*k, 3, stride=1), nn.ReLU(),
            Flatten(), nn.Linear(1024, 10), nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.Linear(10, 1024), nn.ReLU(),
            Deflatten(4*k),
            nn.ConvTranspose2d(4*k, 2*k, 3, stride=1), nn.ReLU(),
            nn.ConvTranspose2d(2*k, k, 3, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(k, 1, 3, stride=2, output_padding=1), nn.Sigmoid(),
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
# 모델, 손실 함수, 최적화 기법 정의하기
model = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# 학습하기
# 합성곱 오토인코더는 이미지 처리에 있어서 기본 신경망을 사용하는 오토인코더보다 성능이 뛰어나다.
for epoch in range(51):
    running_loss = 0.0
    for data in trainloader:
        inputs = data[0].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(inputs, outputs)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    cost = running_loss / len(trainloader)
    if epoch % 10 == 0:
        print("[%d] loss : %.3f"%(epoch+1, cost))